In [ ]:
#!/usr/bin/python

import cv2
import numpy as np
import random
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from skimage import data
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh
import time
from numba import jit
import os,psutil

%matplotlib inline

image=cv2.imread('zoomed.jpg',0)
frame=cv2.imread('zoomed.jpg')
    
# Transform to HSV
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
# Apply Filter to detect the blue side panels of the gates
lower_red = np.array([29,41,0])
upper_red = np.array([88,86,145])

lower_red_line = np.array([62,0,47])
upper_red_line = np.array([91,255,208])

mask = cv2.inRange(hsv, lower_red, upper_red)
res  =  cv2.bitwise_and(frame,frame, mask= mask)
mask = cv2.blur(mask,(20,20))  


mask_line = cv2.inRange(hsv, lower_red_line, upper_red_line)

mask_line = cv2.blur(mask_line,(10,10)) 
mask_line = cv2.blur(mask_line,(10,10))
mask_line = cv2.blur(mask_line,(10,10))


kernel = np.ones((5,5),np.uint8)
mask_line = cv2.erode(mask_line,kernel,iterations = 2)

edges = cv2.Canny(mask_line,100,150,apertureSize = 3)

lines = cv2.HoughLines(edges,1,np.pi/180,100)
lines
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(mask_line,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imwrite('houghlines3.jpg',mask_line)






blobs_dog = blob_dog(mask, max_sigma=10, threshold=.5)


blobs_list = [blobs_dog]
colors = ['red']
titles = [ 'Blob Detections']
     #         'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)
   
fig, axes = plt.subplots(1, 3, figsize=(60, 30), sharex=True, sharey=True)
fig.delaxes(axes.flatten()[2])
fig.delaxes(axes.flatten()[1])
ax = axes.ravel()

cc=0;
for idx, (blobs, color, title) in enumerate(sequence):
    pp=np.ones((len(blobs),2))
    ax[idx].set_title(title)
    ax[idx].imshow(frame)
    for blob in blobs:
        y, x, r = blob
        pp[cc,1]=y;
        pp[cc,0]=x;
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
        cc=cc+1;
    ax[idx].set_axis_off()

# Check if a point is inside a rectangle
def rect_contains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[2] :
        return False
    elif point[1] > rect[3] :
        return False
    return True

# Draw a point
def draw_point(img, p, color ) :
    cv2.circle( img, p, 2, color, cv2.FILLED, cv2.LINE_AA, 0 )


# Draw delaunay triangles
def draw_delaunay(img, subdiv, delaunay_color ) :

    triangleList = subdiv.getTriangleList();
    size = img.shape
    r = (0, 0, size[1], size[0])

    for t in triangleList :
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        
        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3) :
        
            cv2.line(img, pt1, pt2, delaunay_color, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt2, pt3, delaunay_color, 1, cv2.LINE_AA, 0)
            cv2.line(img, pt3, pt1, delaunay_color, 1, cv2.LINE_AA, 0)


# Draw voronoi diagram
def draw_voronoi(img, subdiv) :

    ( facets, centers) = subdiv.getVoronoiFacetList([])

    for i in range(0,len(facets)) :
        ifacet_arr = []
        for f in facets[i] :
            ifacet_arr.append(f)
        
        ifacet = np.array(ifacet_arr, np.int)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        cv2.fillConvexPoly(img, ifacet, color, cv2.LINE_AA, 0);
        ifacets = np.array([ifacet])
        cv2.polylines(img, ifacets, True, (0, 0, 0), 1, cv2.LINE_AA, 0)
        cv2.circle(img, (centers[i][0], centers[i][1]), 3, (0, 0, 0), cv2.FILLED, cv2.LINE_AA, 0)


if __name__ == '__main__':

    # Define window names
    win_delaunay = "Delaunay Triangulation"
    win_voronoi = "Voronoi Diagram"

    # Turn on animation while drawing triangles
    animate = True
    
    # Define colors for drawing.
    delaunay_color = (255,255,255)
    points_color = (0, 0, 255)

    # Read in the image.
    img = cv2.imread("obama.jpg");
    
    # Keep a copy around
    img_orig = img.copy();
    
    # Rectangle to be used with Subdiv2D
    size = img.shape
    rect = (0, 0, size[1], size[0])
    
    # Create an instance of Subdiv2D
    subdiv = cv2.Subdiv2D(rect);

    # Create an array of points.
    points = [];
    
    # Read in the points from a text file
    with open("obama.txt") as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))

    # Insert points into subdiv
    for p in points :
        subdiv.insert(p)
        
        # Show animation
        if animate :
            img_copy = img_orig.copy()
            # Draw delaunay triangles
            draw_delaunay( img_copy, subdiv, (255, 255, 255) );
            cv2.imshow(win_delaunay, img_copy)
            cv2.waitKey(100)

    # Draw delaunay triangles
    draw_delaunay( img, subdiv, (255, 255, 255) );

    # Draw points
    for p in points :
        draw_point(img, p, (0,0,255))

    # Allocate space for voronoi Diagram
    img_voronoi = np.zeros(img.shape, dtype = img.dtype)

    # Draw voronoi diagram
    draw_voronoi(img_voronoi,subdiv)

    # Show results
    cv2.imshow(win_delaunay,img)
    cv2.imshow(win_voronoi,img_voronoi)
    cv2.waitKey(0)